# AwkwardArrays in Julia for High-Energy Physics Data Analysis

AwkwardArrays are designed to handle irregular, nested data structures with Python being the primary language

 * [Reading RNTuple data with Uproot](https://ariostas-talks.github.io/2024-07-02-pyhep-uproot-rntuple/lab/index.html) by Andres Rios-Tascon
 * [Distributed Columnar HEP analysis using coffea + dask](https://github.com/ikrommyd/pyhep2024-coffea-dask) by Iason Krommydas
 * Easy Columnar File Conversion with ‘hepconvert' by Zoë Bilodeau
 * A new SymPy backend for vector: uniting experimental and theoretical physicists by Saransh Chopra

Recent integration into Julia offers new possibilities

## Installation and Setup

PythonCall and Julia Call allow to call Python code from Julia and Julia code from Python via a symmetric interface.

In [1]:
pip install juliacall

Note: you may need to restart the kernel to use updated packages.


In [2]:
from juliacall import Main as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
